In [6]:
try:
    import cPickle as pickle
except ModuleNotFoundError:
    import pickle

from kaveh.behavioral.oculomotor.session import session
from kaveh.toolbox import find_file
from neo.io import Spike2IO
from matplotlib import pyplot as plt
import numpy as np

import numpy as np
from matplotlib import pyplot as plt
# from neo.io import Spike2IO
from kaveh.toolbox import closest_argmin, find_file
from smr import File
from kaveh.plots import axvlines
import os
from kaveh.sorting.spikesorter import SimpleSpikeSorter
import sys

from neo import Spike2IO
from sklearn.mixture import GaussianMixture
import csv
import fnmatch

In [5]:
with open('../data/neurons_david_data_with_spike_train.csv', 'r') as csv_f:
    reader = csv.reader(csv_f)
    csv_content = np.array(list(reader))
file_names = np.array([fn[0] for fn in csv_content])

for i, fn in enumerate(file_names):
    print('{}: {}'.format(i, fn))

0: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O22/O22_2Adapt_BW5L_FW5LCFRDone.smr
1: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O22/O22_2PreAdaptCFRDone.smr
2: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_adapt40pct.smr
3: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_adapt30pct.smr
4: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_preadapt.smr
5: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O13/O13_3_adapt40pctB.smr
6: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O41/O41_1_FW5L_BW5R_A.smr
7: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O41/O41_1_pre.smr
8: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O29/O29_4Adapt_FW5L_BW5RCFR611sec.smr
9: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O29/O29_4PreAdaptCFRDone.smr
10: /mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K14/error_direction/K14_1_directiona

In [7]:
with open('../data/david_neurs.csv', 'r') as csv_f:
    reader = csv.reader(csv_f)
    csv_content = np.array(list(reader))
neuron_names = np.array([fn[0].split('\t')[0].split('\t')[0] for fn in csv_content if fn[0][-1][-1] == '1'])

file_names = []
for nn in neuron_names:
    matches = []
    for root, dirnames, filenames in os.walk('../data/david_neurons/' + nn):
        for filename in fnmatch.filter(filenames, '*.smr.pkl'):
            matches.append(os.path.join(root, filename))
    if matches:
        for ff in matches:
            if not os.path.exists(ff+'.session.pkl'):
                file_names.append(os.path.basename(ff)[0:-4])
                
                

In [8]:
for ff in file_names:

    try:
        f_name = find_file(ff, '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/')
        f_name_tacdec = find_file(ff +'.pkl', '../data/david_neurons')
        print(f_name)
        print(f_name_tacdec)
        neo_reader = Spike2IO(filename=f_name)
        neo_data = neo_reader.read()
        data_block = neo_data[0]
        seg = data_block.segments[0]

            # Load eye data
        HE = seg.analogsignals[0].as_array()
        t_HE = seg.analogsignals[0].times;

        VE = seg.analogsignals[1].as_array()
        t_VE = seg.analogsignals[1].times

        HT = seg.analogsignals[2].as_array()
        t_HT = seg.analogsignals[2].times

        VT = seg.analogsignals[3].as_array()
        t_VT = seg.analogsignals[3].times

        fs = seg.analogsignals[0].sampling_rate
        dt = seg.analogsignals[0].sampling_period

        t_start = 0
        t_end = 20000
        mysess = session(HT[np.where(np.logical_and((t_HT > t_start), (t_HT <= t_end)))], t_HT[np.where(np.logical_and((t_HT > t_start), (t_HT <= t_end)))],
                         VT[np.where(np.logical_and((t_VT > t_start), (t_VT <= t_end)))], t_VT[np.where(np.logical_and((t_VT > t_start), (t_VT <= t_end)))],
                         HE[np.where(np.logical_and((t_HE > t_start), (t_HE <= t_end)))], t_HE[np.where(np.logical_and((t_HE > t_start), (t_HE <= t_end)))],
                         VE[np.where(np.logical_and((t_VE > t_start), (t_VE <= t_end)))], t_VE[np.where(np.logical_and((t_VE > t_start), (t_VE <= t_end)))], fs, dt)
        mysess._cut_to_min_size()
        mysess._calc_target_velocity()
        mysess._calc_saccade_velocity()
        mysess._detect_target_jumps()
        mysess._detect_saccades()
        mysess._calc_error_vectors()

        # saving mysess
        output_fn = f_name_tacdec + '.session.pkl'
        print('writing')
        print(output_fn)
        with open(output_fn, 'wb') as output:
            print('writing {} ...'.format(output_fn))
            pickle.dump(mysess, output, pickle.HIGHEST_PROTOCOL)

    except Exception as e: 
        print(e)
        pass
        
    

/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Pip/P21/P21_1 LBW5 RFW5 D.smr
../data/david_neurons/P21/P21_1 LBW5 RFW5 D.smr.pkl


/usr/local/lib/python2.7/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


writing
../data/david_neurons/P21/P21_1 LBW5 RFW5 D.smr.pkl.session.pkl
writing ../data/david_neurons/P21/P21_1 LBW5 RFW5 D.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Pip/P21/P21_1 LFW5 RBW5 C.smr
../data/david_neurons/P21/P21_1 LFW5 RBW5 C.smr.pkl


writing
../data/david_neurons/P21/P21_1 LFW5 RBW5 C.smr.pkl.session.pkl
writing ../data/david_neurons/P21/P21_1 LFW5 RBW5 C.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Pip/P21/P21_1 pre A.smr
../data/david_neurons/P21/P21_1 pre A.smr.pkl


writing
../data/david_neurons/P21/P21_1 pre A.smr.pkl.session.pkl
writing ../data/david_neurons/P21/P21_1 pre A.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Pip/P21/P21_1 LBW5 RFW5 B.smr
../data/david_neurons/P21/P21_1 LBW5 RFW5 B.smr.pkl


...

writing
../data/david_neurons/O60/O60_1_FW5L_BW5R_D.smr.pkl.session.pkl
writing ../data/david_neurons/O60/O60_1_FW5L_BW5R_D.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O60/O60_1_FW5L_BW5R_B.smr
../data/david_neurons/O60/O60_1_FW5L_BW5R_B.smr.pkl


writing
../data/david_neurons/O60/O60_1_FW5L_BW5R_B.smr.pkl.session.pkl
writing ../data/david_neurons/O60/O60_1_FW5L_BW5R_B.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O60/O60_1_FW3L_BW3R_A.smr
../data/david_neurons/O60/O60_1_FW3L_BW3R_A.smr.pkl


writing
../data/david_neurons/O60/O60_1_FW3L_BW3R_A.smr.pkl.session.pkl
writing ../data/david_neurons/O60/O60_1_FW3L_BW3R_A.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2006/Oscar/O83/O83_1_mix_N_FW5L_BW5R_A.smr
../data/david_neurons/O84/O83_1_mix_N_FW5L_BW5R_A.smr.pkl


writing
../data/david_neurons/O84/O83_1_mix_N_FW5L_BW5R_A.smr.pkl.session.pkl
writing ../data/david_neurons/O84/O83_1_mix_N_FW5L_BW5R_A.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K32/K32_1control.smr
../data/david_neurons/K32/K32_1control.smr.pkl
writing
../data/david_neurons/K32/K32_1control.smr.pkl.session.pkl
writing ../data/david_neurons/K32/K32_1control.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K32/K32_1oblRBWLBWadapt.smr
../data/david_neurons/K32/K32_1oblRBWLBWadapt.smr.pkl
writing
../data/david_neurons/K32/K32_1oblRBWLBWadapt.smr.pkl.session.pkl
writing ../data/david_neurons/K32/K32_1oblRBWLBWadapt.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K32/error_direction/K32_1dirtest.smr
../data/david_neurons/K32/error_direction/K32_1dirtest.smr.pkl
writing
../data/david_neurons/K32/error_direction/K32_1dirtest.smr.pkl.session.pkl
writing ../data/david_

writing
../data/david_neurons/K50/error_magnitude/K50_2_Horzerrorsizetest_leveltriggerable.smr.pkl.session.pkl
writing ../data/david_neurons/K50/error_magnitude/K50_2_Horzerrorsizetest_leveltriggerable.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K34/K34_1HadaptbothBW.smr
../data/david_neurons/K34/K34_1HadaptbothBW.smr.pkl
writing
../data/david_neurons/K34/K34_1HadaptbothBW.smr.pkl.session.pkl
writing ../data/david_neurons/K34/K34_1HadaptbothBW.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K34/K34_1control.smr
../data/david_neurons/K34/K34_1control.smr.pkl
writing
../data/david_neurons/K34/K34_1control.smr.pkl.session.pkl
writing ../data/david_neurons/K34/K34_1control.smr.pkl.session.pkl ...
/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2008_Random/Kimo/K34/K34_1SStest_small_large_sacccades.smr
../data/david_neurons/K34/K34_1SStest_small_large_sacccades.smr.pkl
writing
../data/david_neurons/K34/K34

In [3]:
file_names[61]

'/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Felix/F102506/F102506_1506_List.smr'

In [3]:
ff = file_names[63]
f_name = find_file(os.path.basename(ff), '/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/')
f_name_tacdec = find_file(os.path.basename(ff)+'.pkl', '../data/david_neurons')
print(f_name)
print(f_name_tacdec)
neo_reader = Spike2IO(filename=f_name)
neo_data = neo_reader.read()
data_block = neo_data[0]
seg = data_block.segments[0]

    # Load eye data
HE = seg.analogsignals[0].as_array()
t_HE = seg.analogsignals[0].times;

VE = seg.analogsignals[1].as_array()
t_VE = seg.analogsignals[1].times

HT = seg.analogsignals[2].as_array()
t_HT = seg.analogsignals[2].times

VT = seg.analogsignals[3].as_array()
t_VT = seg.analogsignals[3].times

fs = seg.analogsignals[0].sampling_rate
dt = seg.analogsignals[0].sampling_period

t_start = 0
t_end = 20000
mysess = session(HT[np.where(np.logical_and((t_HT > t_start), (t_HT <= t_end)))], t_HT[np.where(np.logical_and((t_HT > t_start), (t_HT <= t_end)))],
                 VT[np.where(np.logical_and((t_VT > t_start), (t_VT <= t_end)))], t_VT[np.where(np.logical_and((t_VT > t_start), (t_VT <= t_end)))],
                 HE[np.where(np.logical_and((t_HE > t_start), (t_HE <= t_end)))], t_HE[np.where(np.logical_and((t_HE > t_start), (t_HE <= t_end)))],
                 VE[np.where(np.logical_and((t_VE > t_start), (t_VE <= t_end)))], t_VE[np.where(np.logical_and((t_VE > t_start), (t_VE <= t_end)))], fs, dt)
mysess._cut_to_min_size()
mysess._calc_target_velocity()
mysess._calc_saccade_velocity()
mysess._detect_target_jumps()
mysess._detect_saccades()
mysess._calc_error_vectors()

# saving mysess
output_fn = f_name_tacdec + '.session.pkl'
print('writing')
print(output_fn)
with open(output_fn, 'wb') as output:
    print('writing {} ...'.format(output_fn))
    pickle.dump(mysess, output, pickle.HIGHEST_PROTOCOL)

/mnt/papers/Herzfeld_Nat_Neurosci_2018/raw_data/2010_Adapt/Felix/F112006/F112006_1446_List.smr
../data/david_neurons/F112006/F112006_1446_List.smr.pkl


/usr/local/lib/python2.7/dist-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


writing
../data/david_neurons/F112006/F112006_1446_List.smr.pkl.session.pkl
writing ../data/david_neurons/F112006/F112006_1446_List.smr.pkl.session.pkl ...


In [13]:
[i for i, x in enumerate(mysess.saccade_onsets) if x is None]


[0]